In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D

import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data

from tqdm import tqdm
from sklearn import datasets
import random

In [3]:
import mylibrary.datasets as datasets

In [4]:
device = torch.device("cuda:0")
# device = torch.device("cuda:1")
# device = torch.device("cpu")

In [5]:
mnist = datasets.FashionMNIST()
# mnist.download_mnist()
# mnist.save_mnist()
train_data, train_label_, test_data, test_label_ = mnist.load()

train_data = train_data / 255.
test_data = test_data / 255.

train_size = len(train_label_)

In [6]:
## converting data to pytorch format
train_data = torch.Tensor(train_data)
test_data = torch.Tensor(test_data)
train_label = torch.LongTensor(train_label_)
test_label = torch.LongTensor(test_label_)

In [7]:
## converting data to pytorch format
train_data = torch.Tensor(train_data)
test_data = torch.Tensor(test_data)
train_label = torch.LongTensor(train_label_)

In [8]:
input_size = 784
output_size = 10

In [9]:
class MNIST_Dataset(data.Dataset):
    
    def __init__(self, data, label):
        self.data = data
        self.label = label
        
#         self.label = mask.type(torch.float32).reshape(-1,1)
        self._shuffle_data_()
        
    def __len__(self):
        return len(self.data)
    
    def _shuffle_data_(self):
        randidx = random.sample(range(len(self.data)), k=len(self.data))
        self.data = self.data[randidx]
        self.label = self.label[randidx]
    
    def __getitem__(self, idx):
        img, lbl = self.data[idx], self.label[idx]
        return img, lbl

In [10]:
train_dataset = MNIST_Dataset(train_data, train_label)
test_dataset = MNIST_Dataset(test_data, test_label)

In [11]:
learning_rate = 0.0003
batch_size = 50

In [12]:
train_loader = data.DataLoader(dataset=train_dataset, num_workers=4, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(dataset=test_dataset, num_workers=4, batch_size=batch_size, shuffle=False)

## Making of matrix factorized layer

In [13]:
class PairBilinear(nn.Module):
    def __init__(self, dim, grid_width):
        super().__init__()
        self.dim = dim
        self.grid_width = grid_width
        
        self.num_pairs = self.dim // 2
        along_row = torch.linspace(0, 1, self.grid_width).reshape(1, -1)
        along_col = torch.linspace(0, 1, self.grid_width).reshape(-1, 1)
        self.Y = torch.stack([along_row+along_col*0, along_row*0+along_col])
        self.Y = torch.repeat_interleave(self.Y.unsqueeze(0), self.num_pairs, dim=0)
        self.Y = nn.Parameter(self.Y)
        
        self.pairW = torch.eye(2).unsqueeze(0).repeat_interleave(self.num_pairs, dim=0)*0.5
        self.pairW = nn.Parameter(self.pairW)
    
    def forward(self, x):
        bs = x.shape[0]
        x = x.view(-1, 2)
        
        _wi = torch.arange(self.num_pairs).repeat(bs)
        _W = self.pairW[_wi]
        x = torch.bmm(x.unsqueeze(1), _W).squeeze(1)
        
        x = x*self.grid_width
        index = torch.clamp(x.data, 0, self.grid_width-2)
        index = torch.floor(index)
        x = x-index
        
        index = (index.repeat_interleave(2, dim=0))
        
        _bi = torch.arange(bs).repeat_interleave(self.num_pairs*2)
        _gi = torch.arange(self.num_pairs).repeat_interleave(2).repeat(bs)
        _pi = torch.LongTensor([0,1]).repeat(bs*self.num_pairs)
        _xc, _yc = tuple(index.type(torch.long).t())

        f00 = self.Y[_gi, _pi, _xc, _yc]
        f01 = self.Y[_gi, _pi, _xc, _yc+1]
        f10 = self.Y[_gi, _pi, _xc+1, _yc]
        f11 = self.Y[_gi, _pi, _xc+1, _yc+1]
        
        a00 = f00
        a10 = f10-f00
        a01 = f01-f00
        a11 = f11-f10-f01+f00
        
        ##### this doubles the multiplication for x,y
#         x = x.repeat_interleave(2, dim=0)
#         y = a00 + x[:, 0]*a10 + x[:, 1]*a01 + x[:, 0]*x[:, 1]*a11

        ##### this repeats in individual way
        _x, _y = x[:, 0].repeat_interleave(2), x[:, 1].repeat_interleave(2)
        y = a00 + _x*a10 + _y*a01 + _x*_y*a11

        ### now 
        y = y.view(bs, -1)
        return y

In [24]:
class BiasLayer(nn.Module):
    def __init__(self, dim, init_val=0):
        super().__init__()
        self.bias = nn.Parameter(torch.ones(dim)*init_val)
        
    def forward(self, x):
        return x+self.bias

In [25]:
class FactorizedPairBilinearSpline(nn.Module):
    
    def __init__(self, input_dim, grid_width):
        super().__init__()
        assert input_dim%2 == 0, "Input dim must be even number"
        self.input_dim = input_dim
        num_layers = int(np.ceil(np.log2(8)))
            
        self.facto_nets = []
        self.idx_revidx = []
        for i in range(num_layers):
            idrid = self.get_pair(self.input_dim, i+1)
            net = PairBilinear(self.input_dim, grid_width)
            self.facto_nets.append(net)
            self.idx_revidx.append(idrid)
        self.facto_nets = nn.ModuleList(self.facto_nets)
            
    def get_pair(self, inp_dim, step=1):
        dim = 2**int(np.ceil(np.log2(inp_dim)))
        assert isinstance(step, int), "Step must be integer"

        blocks = (2**step)
        range_ = dim//blocks
        adder_ = torch.arange(0, range_)*blocks

        pairs_ = torch.Tensor([0, blocks//2])
        repeat_ = torch.arange(0, blocks//2).reshape(-1,1)
        block_map = (pairs_+repeat_).reshape(-1)

        reorder_for_pair = (block_map+adder_.reshape(-1,1)).reshape(-1)
        indx = reorder_for_pair.type(torch.long)
        indx = indx[indx<inp_dim]

        rev_indx = torch.argsort(indx)
        return indx, rev_indx
    
    def forward(self, x):
        ## swap first and then forward and reverse-swap
        y = x
        for i in range(len(self.facto_nets)):
            idx, revidx = self.idx_revidx[i]
            y = y[:, idx]
            y = self.facto_nets[i](y) 
            y = y[:, revidx]
        y = x + y ## this is residual addition... remove if only want feed forward
        return y

In [26]:
pfL = FactorizedPairBilinearSpline(100, 5)

In [27]:
pfL(torch.randn(2, 100))

tensor([[-2.5868e+00,  6.7256e-01, -1.0017e+00,  1.1068e+00,  8.0416e-04,
          3.7501e-01, -7.0494e-01, -8.0167e-01, -2.3931e+00, -6.1151e-01,
          5.2585e-01,  3.8225e-01, -1.1707e+00, -2.3331e+00, -5.5769e-01,
          4.1153e-01,  3.0017e-01, -1.6302e-01,  6.9280e-02,  1.1011e+00,
         -7.6341e-01, -2.1690e-01,  2.5648e-01,  1.2808e+00,  7.6862e-01,
         -1.2344e+00,  7.4107e-01, -1.1313e+00,  2.3732e-01,  4.6992e-01,
          4.5366e-01, -2.6975e-02, -9.9379e-01, -2.3096e-01,  2.0784e-01,
         -2.0274e+00, -1.1636e+00, -7.7709e-01,  7.3316e-01,  1.1328e+00,
         -8.1261e-01, -8.7045e-01, -4.5991e-01, -6.1483e-01,  1.3671e-01,
          6.2548e-01, -6.5829e-01, -2.0479e+00,  4.1311e-01,  9.9249e-02,
         -6.8635e-01,  7.4326e-01,  6.0960e-01,  8.8812e-02,  5.6190e-02,
          3.1242e-01,  1.9525e+00, -1.2611e-01,  1.0271e+00, -1.6453e+00,
         -2.2031e-01,  3.0264e-01,  5.7827e-01, -6.1048e-02, -6.5802e-01,
          4.5158e-01,  1.2633e+00, -3.

In [28]:
param_count = sum([torch.numel(p) for p in pfL.parameters()])
param_count

8100

In [43]:
class FactorNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.bias = BiasLayer(784)
        self.la1 = FactorizedPairBilinearSpline(784, grid_width=5)
        self.bn1 = nn.BatchNorm1d(784)
        self.fc = nn.Linear(784, 10)
        
    def forward(self, x):
        x = self.bias(x)
        x = self.bn1(self.la1(x))
        x = self.fc(x)
        return x

In [44]:
# class OrdinaryNet(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.la1 = nn.Linear(784, 1024, bias=False)
#         self.bn1 = nn.BatchNorm1d(1024)
#         self.la2 = nn.Linear(1024, 1024)
#         self.bn2 = nn.BatchNorm1d(1024, bias=False)
#         self.la3 = nn.Linear(1024, 10)
        
#     def forward(self, x):
#         x = self.bn1(self.la1(x))
#         x = torch.relu(x)
#         x = self.bn2(self.la2(x))
#         x = torch.relu(x)
#         x = self.la3(x)
#         return x

In [45]:
model = FactorNet()
param_count = sum([torch.numel(p) for p in model.parameters()])
param_count

73706

In [46]:
# model = OrdinaryNet()
# param_count1 = sum([torch.numel(p) for p in model.parameters()])
# param_count1, param_count1/param_count

### Model Development

In [47]:
torch.manual_seed(0)
model = FactorNet().to(device)
# model = OrdinaryNet().to(device)
model

AssertionError: Torch not compiled with CUDA enabled

In [27]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [28]:
print("number of params: ", sum(p.numel() for p in model.parameters()))

number of params:  1867786


In [29]:
losses = []
train_accs = []
test_accs = []
EPOCHS = 20

for epoch in range(EPOCHS):
    
    train_acc = 0
    train_count = 0
    for xx, yy in tqdm(train_loader):
        xx, yy = xx.to(device), yy.to(device)

        yout = model(xx)
        loss = criterion(yout, yy)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(float(loss))

        outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
        correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
        train_acc += correct
        train_count += len(outputs)

    train_accs.append(float(train_acc)/train_count*100)
    train_acc = 0
    train_count = 0

    print(f'Epoch: {epoch},  Loss:{float(loss)}')
    test_count = 0
    test_acc = 0
    for xx, yy in tqdm(test_loader):
        xx, yy = xx.to(device), yy.to(device)
        with torch.no_grad():
            yout = model(xx)
        outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
        correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
        test_acc += correct
        test_count += len(xx)
    test_accs.append(float(test_acc)/test_count*100)
    print(f'Train Acc:{train_accs[-1]:.2f}%, Test Acc:{test_accs[-1]:.2f}%')
    print()

### after each class index is finished training
print(f'\t-> Train Acc {max(train_accs)} ; Test Acc {max(test_accs)}')

  0%|          | 0/200 [00:00<?, ?it/s]

Epoch: 0,  Loss:0.4663251042366028


  0%|          | 0/1200 [00:00<?, ?it/s]

Train Acc:84.99%, Test Acc:86.40%



  0%|          | 0/200 [00:00<?, ?it/s]

...

Train Acc:97.94%, Test Acc:89.55%



  0%|          | 0/200 [00:00<?, ?it/s]

Epoch: 19,  Loss:0.027473196387290955


100%|██████████| 200/200 [00:00<00:00, 651.95it/s]

Train Acc:97.96%, Test Acc:89.13%

	-> Train Acc 97.95833333333334 ; Test Acc 89.61


In [ ]:
## stats: 20 epochs
### for factor-net -> 57354 -> Train Acc 88.83666666666666 ; Test Acc 86.7
### for ordinary-net -> 1867786 -> Train Acc 97.95833333333334 ; Test Acc 89.61